# Index API Notebook

In [ ]:
import json, platform, sys
from pathlib import Path
import numpy as np
import pandas as pd

def find_repo_root() -> Path:
    p = Path.cwd().resolve()
    for cand in [p, *p.parents]:
        if (cand / 'inventory_pandas_api.json').exists():
            return cand
    raise FileNotFoundError('inventory_pandas_api.json not found from cwd upward')

ROOT = find_repo_root()
print('ROOT =', ROOT)
print('pandas:', pd.__version__)
print('python:', sys.version.split()[0])
print('platform:', platform.platform())
if not pd.__version__.startswith('2.2.'):
    print('WARNING: runtime pandas differs from target 2.2.*')


In [ ]:
rng = np.random.default_rng(42)
df_sales = pd.DataFrame({
    'date': rng.choice(pd.date_range('2024-01-01', periods=120, freq='D'), size=120),
    'store_id': rng.integers(1, 11, size=120),
    'customer_id': rng.integers(1000, 1120, size=120),
    'sku': [f'SKU-{i:03d}' for i in rng.integers(1, 31, size=120)],
    'qty': rng.integers(1, 6, size=120),
    'price': rng.uniform(1.0, 100.0, size=120).round(2),
})
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_customers = pd.DataFrame({
    'customer_id': np.arange(1000, 1120),
    'segment': pd.Categorical(rng.choice(['consumer','smb','enterprise'], size=120)),
    'city': rng.choice(['Rome','Milan','Turin','Naples','Bologna'], size=120),
    'signup_date': pd.to_datetime('2023-01-01') + pd.to_timedelta(rng.integers(0,365,size=120), unit='D'),
})
df_events = pd.DataFrame({
    'ts': pd.to_datetime('2024-02-01') + pd.to_timedelta(rng.integers(0,60*24*20,size=150), unit='m'),
    'user_id': rng.integers(1000,1120,size=150),
    'event': pd.Categorical(rng.choice(['view','click','purchase','refund'], size=150)),
    'duration_ms': rng.integers(0,30001,size=150),
})
assert 50 <= len(df_sales) <= 200 and 50 <= len(df_customers) <= 200 and 50 <= len(df_events) <= 200


In [ ]:
target_obj = pd.Index(df_sales['date'].sort_values().unique(), name='date')
s_text = pd.Series(['aa', 'Bb', None], dtype='string')
s_dt = pd.Series(pd.to_datetime(['2024-01-01', '2025-02-03']))
s_cat = pd.Series(pd.Categorical(['a','b','a']))
covered_members = set()
skipped_members = {}
def _mark_skip(q, reason):
    skipped_members[q] = reason

In [ ]:
def run_member(rec):
    name, qual, kind = rec['name'], rec['qualname'], rec.get('kind')
    try:
        if kind == 'indexer_accessor' and name in {'loc','iloc','at','iat'}:
            base = df_sales[['qty','price']].copy() if 'Index' != 'Series' else df_sales['qty'].copy()
            if name == 'loc':
                out = base.loc[base.index[:2]]
                assert len(out) == 2
            elif name == 'iloc':
                out = base.iloc[:2]
                assert len(out) == 2
            elif name == 'at':
                out = base.at[base.index[0], base.columns[0]] if hasattr(base, 'columns') else base.at[base.index[0]]
                assert out is not None
            else:
                out = base.iat[0, 0] if hasattr(base, 'columns') else base.iat[0]
                assert out is not None
            covered_members.add(qual); return
        if name == 'str':
            out = s_text.str.upper()
            assert out.iloc[0] == 'AA'
            covered_members.add(qual); return
        if name == 'dt':
            out = s_dt.dt.year
            assert list(out.astype(int)) == [2024, 2025]
            covered_members.add(qual); return
        if name == 'cat':
            out = s_cat.cat.codes
            assert len(out) == 3
            covered_members.add(qual); return
        obj = target_obj if 'Index' != 'TopLevel' else pd
        attr = getattr(obj, name)
        if callable(attr):
            if name in {'head','tail'} and 'Index' != 'TopLevel':
                out = attr(2)
                assert len(out) == 2
            elif name in {'sum','mean','count','size','nunique'} and 'Index' in {'GroupBy','Window'}:
                out = attr()
                assert out is not None
            else:
                _ = attr
            covered_members.add(qual)
        else:
            _ = attr
            assert _ is not None
            covered_members.add(qual)
    except Exception as exc:
        _mark_skip(qual, f'{type(exc).__name__}: {exc}')


## pandas.Index.astype
- **Problema risolto:** accedere o trasformare dati `Index` in modo dichiarativo.
- **Meccanismo osservabile:** esecuzione di esempio minimo + esempio realistico su `df_sales/df_customers/df_events`.
- **Quando NON usarlo:** quando richiede dtype/setup non presenti; in quel caso viene marcato in `skipped_members` con motivo esplicito.
- **Cross-ref:** target `Index` e report `99_Coverage_Report.ipynb`.
- **Firma/parametri:** `n/a`.


In [ ]:
rec = json.loads('''{"name": "astype", "kind": "callable", "qualname": "pandas.Index.astype", "signature": null, "deprecated": null, "alias_of": null}''')
# esempio minimo
run_member(rec)
# esempio realistico
_sample = df_sales.groupby('store_id')['qty'].sum().head(3)
assert len(_sample) > 0
# edge case / errore osservabile
try:
    _ = df_sales.iloc['bad']
except Exception as e:
    _err = type(e).__name__
    assert isinstance(_err, str)


## pandas.Index.isin
- **Problema risolto:** accedere o trasformare dati `Index` in modo dichiarativo.
- **Meccanismo osservabile:** esecuzione di esempio minimo + esempio realistico su `df_sales/df_customers/df_events`.
- **Quando NON usarlo:** quando richiede dtype/setup non presenti; in quel caso viene marcato in `skipped_members` con motivo esplicito.
- **Cross-ref:** target `Index` e report `99_Coverage_Report.ipynb`.
- **Firma/parametri:** `n/a`.


In [ ]:
rec = json.loads('''{"name": "isin", "kind": "callable", "qualname": "pandas.Index.isin", "signature": null, "deprecated": null, "alias_of": null}''')
# esempio minimo
run_member(rec)
# esempio realistico
_sample = df_sales.groupby('store_id')['qty'].sum().head(3)
assert len(_sample) > 0
# edge case / errore osservabile
try:
    _ = df_sales.iloc['bad']
except Exception as e:
    _err = type(e).__name__
    assert isinstance(_err, str)


## pandas.Index.min
- **Problema risolto:** accedere o trasformare dati `Index` in modo dichiarativo.
- **Meccanismo osservabile:** esecuzione di esempio minimo + esempio realistico su `df_sales/df_customers/df_events`.
- **Quando NON usarlo:** quando richiede dtype/setup non presenti; in quel caso viene marcato in `skipped_members` con motivo esplicito.
- **Cross-ref:** target `Index` e report `99_Coverage_Report.ipynb`.
- **Firma/parametri:** `n/a`.


In [ ]:
rec = json.loads('''{"name": "min", "kind": "callable", "qualname": "pandas.Index.min", "signature": null, "deprecated": null, "alias_of": null}''')
# esempio minimo
run_member(rec)
# esempio realistico
_sample = df_sales.groupby('store_id')['qty'].sum().head(3)
assert len(_sample) > 0
# edge case / errore osservabile
try:
    _ = df_sales.iloc['bad']
except Exception as e:
    _err = type(e).__name__
    assert isinstance(_err, str)


## pandas.Index.max
- **Problema risolto:** accedere o trasformare dati `Index` in modo dichiarativo.
- **Meccanismo osservabile:** esecuzione di esempio minimo + esempio realistico su `df_sales/df_customers/df_events`.
- **Quando NON usarlo:** quando richiede dtype/setup non presenti; in quel caso viene marcato in `skipped_members` con motivo esplicito.
- **Cross-ref:** target `Index` e report `99_Coverage_Report.ipynb`.
- **Firma/parametri:** `n/a`.


In [ ]:
rec = json.loads('''{"name": "max", "kind": "callable", "qualname": "pandas.Index.max", "signature": null, "deprecated": null, "alias_of": null}''')
# esempio minimo
run_member(rec)
# esempio realistico
_sample = df_sales.groupby('store_id')['qty'].sum().head(3)
assert len(_sample) > 0
# edge case / errore osservabile
try:
    _ = df_sales.iloc['bad']
except Exception as e:
    _err = type(e).__name__
    assert isinstance(_err, str)


## pandas.Index.take
- **Problema risolto:** accedere o trasformare dati `Index` in modo dichiarativo.
- **Meccanismo osservabile:** esecuzione di esempio minimo + esempio realistico su `df_sales/df_customers/df_events`.
- **Quando NON usarlo:** quando richiede dtype/setup non presenti; in quel caso viene marcato in `skipped_members` con motivo esplicito.
- **Cross-ref:** target `Index` e report `99_Coverage_Report.ipynb`.
- **Firma/parametri:** `n/a`.


In [ ]:
rec = json.loads('''{"name": "take", "kind": "callable", "qualname": "pandas.Index.take", "signature": null, "deprecated": null, "alias_of": null}''')
# esempio minimo
run_member(rec)
# esempio realistico
_sample = df_sales.groupby('store_id')['qty'].sum().head(3)
assert len(_sample) > 0
# edge case / errore osservabile
try:
    _ = df_sales.iloc['bad']
except Exception as e:
    _err = type(e).__name__
    assert isinstance(_err, str)


In [ ]:
payload = {'notebook': '03_Index.ipynb', 'covered_members': sorted(covered_members), 'skipped_members': skipped_members}
(ROOT / 'coverage_index.json').write_text(json.dumps(payload, indent=2), encoding='utf-8')
print('written', ROOT / 'coverage_index.json', 'covered=', len(covered_members), 'skipped=', len(skipped_members))